# Characterize vesicle from vtu #
The c code keeps breaking, and it doesn't work between versions  
At this point, just reverse the .vtu XML file

The geometry is saved as  

<p style="color:blue;"> &ltVTKfile&gt </p>    

>  \<UnstructuredGrid\>  
> since we have only one vesicle, there is only one piece
>> \<Piece NumberOfPoints="10" NumberOfCells="10"\>  
>> points are the vertices, cells are bonds (2-cells, type 3) and triangles (3-cells, type 5)
>>> \<Points\>  
>>> \<Cells\>  
>>> \<PointData\>  
>>> \<CellData\>  
  
These contain:

<p style="color:blue;"> &ltPoints&gt </p>  

> \<DataArray Name="Koordinate tock"\>  
> contains the coordinates of each vertex  
  
<p style="color:blue;"> &ltCells&gt </p>  

> \<DataArray Name="connectivity"\>  
> connectivity of each cell: 6 8 means "a cell connecting vertex 6 and 8", 10 13 28 means "a cell connecting vertex 10, 13, and 28""  
>  
> \<DataArray Name="offsets"\>  
> excplicitly state the separation in connectivity, rather than relying on the tabs 
>  
> \<DataArray Name="types"\>  
> excplicitly name the type of cell: type 3 is bond, or 2 cell, type 5 is triangle, a 3-cell 

<p style="color:blue;"> &ltPoint/CellData&gt </p>  

> \<DataArray\>s of any other information that is saved:
> curvature, bending energy, etc...
  
    
    
Additionally:  
The \<trisurf\> part of the xml is purely for the c program, as far as I can tell  
additional usefull imformation could be obtained un \<tape\>, which is the just actuall tape file

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
def xml_to_str_list(node):
    ''' Take node, extract text array to list ' 2 3\n4 5 6\n' -> [['2','3'],['4','5','6']'''
    
    all_text=node.text
    
    #trim space and tab prefix and suffix (conda refuses python 3.9)
    for i,t in enumerate(all_text):
        if (t!='\n' and t!=' '):
            break
    all_text=all_text[i:]
    for i,t in enumerate(reversed(all_text)):
        if (t!='\n' and t!=' '):
            break
    if i!=0:
        all_text=all_text[:-i]
    
    #if there are tabs in the middle, assume this is a 2d list
    if any(t=='\n' for t in all_text):
        return [ x.split() for x in all_text.split('\n')]
    else:
        return all_text.split() 

In [3]:
text='  \n  alphn\nsa blag\n  '
for i,t in enumerate(text):
    if (t!='\n' and t!=' '):
        break
text=text[i:]
for i,t in enumerate(reversed(text)):
    if (t!='\n' and t!=' '):
        break
if i!=0:
    text=text[:-i]
if any(t=='\n' for t in text):
    print('ha')
            

ha


In [4]:
#vtu_location="/opt/workspace/msc_project/other/vtu_files/micro_vtu.vtu"
vtu_location="/opt/workspace/msc_project/other/vtu_files/c_plus_minus_20_potato.vtu"

In [5]:
## get the .vtu  xml file and parse it
tree = ET.parse(vtu_location)
root = tree.getroot()

In [6]:
#overall view
print(root.tag)
for child in root:
    print('\t #####################')
    print('\t',child.tag,child.attrib,len(child))
    #for subchild in child:
     #   print('\t',subchild.tag, len(subchild),subchild.attrib)
      #  print('\t\t',subchild.text.split(' '))


VTKFile
	 #####################
	 trisurfversion {} 0
	 #####################
	 dumpdate {} 0
	 #####################
	 tape {} 0
	 #####################
	 trisurf {'nvtx': '2002', 'npoly': '0', 'nmono': '0', 'compressed': 'false'} 4008
	 #####################
	 UnstructuredGrid {} 1


In [7]:
#tape=root.findall('tape')[0]
#print(tape.text)

In [8]:
# get and view the Unstructured grid
ugrid=root.findall('UnstructuredGrid')
vesicle=ugrid[0][0] #"Yer a vesicle, Harry!"
for child in vesicle:
    print('#####')
    print(child.tag,child.attrib,len(child))
    for subchild in child:
        print('\t',subchild.tag,subchild.attrib,len(subchild))

#####
PointData {'Scalars': 'vertices_idx'} 3
	 DataArray {'type': 'Int64', 'Name': 'vertices_idx', 'format': 'ascii'} 0
	 DataArray {'type': 'Float64', 'Name': 'spontaneous_curvature', 'format': 'ascii'} 0
	 DataArray {'type': 'Float64', 'Name': 'bending_energy', 'format': 'ascii'} 0
#####
CellData {} 0
#####
Points {} 1
	 DataArray {'type': 'Float64', 'Name': 'Koordinate tock', 'NumberOfComponents': '3', 'format': 'ascii'} 0
#####
Cells {} 3
	 DataArray {'type': 'Int64', 'Name': 'connectivity', 'format': 'ascii'} 0
	 DataArray {'type': 'Int64', 'Name': 'offsets', 'format': 'ascii'} 0
	 DataArray {'type': 'UInt8', 'Name': 'types', 'format': 'ascii'} 0


In [9]:
#vesicle.findall('Points')[0][0].text
#vesicle.findall('Cells')[0][0].text[:-1]

In [10]:
str_vtx_positions = xml_to_str_list(vesicle.findall('Points/DataArray')[0])
vtx_pos = [ [float(y) for y in x] for x in str_vtx_positions ] 

str_cells = xml_to_str_list(vesicle.findall('Cells/DataArray')[0])
all_cells = [ [int(y) for y in x] for x in str_cells ] 
bonds = [ x for x in all_cells if len(x)==2 ]
triangles = [ x for x in all_cells if len(x)==3 ]
print('example of positions:\n',vtx_pos[6],'\nexamples of bonds:\n',bonds[:5],'\nexamples of triangles:\n',triangles[:3])

example of positions:
 [-8.538495227900299, 5.955783172740956, -11.712382461115256] 
examples of bonds:
 [[1645, 1770], [515, 1461], [1360, 312], [1544, 377], [1930, 1425]] 
examples of triangles:
 [[1885, 560, 1559], [1847, 721, 1534], [1660, 978, 522]]


In [11]:
num_vtx = len(vtx_pos)
num_bonds = len(bonds)
num_triangles = len(triangles)
print(num_vtx,num_bonds,num_triangles)

2002 6000 4000


In [12]:
#root.findall('.//*[@Name="spontaneous_curvature"]')[0].text

In [13]:
c = [ float(x) for x in xml_to_str_list(vesicle.findall('PointData/DataArray')[1])]
benergy = [ float(x) for x in xml_to_str_list(vesicle.findall('PointData/DataArray')[2])]
c[:10], benergy[:10]

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0],
 [0.14711998087372083,
  0.38634867076043977,
  0.5273067918719248,
  0.19358733727431787,
  0.20613498590416948,
  0.4732760812146742,
  0.2684755422924594,
  1.1036729490404547,
  0.011047263925678622,
  0.06831289572825447])

In [14]:
is_curved = np.array(c)!=0
is_curved

adj_mat = np.zeros([num_vtx,num_vtx],dtype=bool)
for bond in bonds:
    adj_mat[bond[0],bond[1]]=True
    adj_mat[bond[1],bond[0]]=True
adj_mat

c_adj_mat = adj_mat.copy()

plt.matshow(adj_mat)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
c_adj_mat[~is_curved,:]=False
c_adj_mat[:,~is_curved]=False
plt.matshow(c_adj_mat)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
graph_c=sp.csgraph.csgraph_from_dense(c_adj_mat)
plt.spy(graph_c,markersize=1)

In [17]:
n, labeled_vtx = sp.csgraph.connected_components(graph_c)
np.bincount(np.bincount(labeled_vtx))

array([   0, 1928,   19,    6,    2,    2])

In [18]:
a=sp.csr_matrix((50,50))
a[1,7]=6
a[7,1]=-8
plt.spy(a)

/home/yoav/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
